In [ ]:
!pip install tensorflow==2.16.0rc0

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
IMAGE_SIZE = 256
CHANNELS = 3

In [ ]:
rom tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True
        )
train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/EthioMedScan/dataset/LC_DataSet/train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=128,
        class_mode="sparse",

)

In [ ]:
train_generator.class_indices

In [ ]:
class_names = list(train_generator.class_indices.keys())
class_names

In [ ]:
count=0
for image_batch, label_batch in train_generator:

    print(image_batch[0])
    break


In [ ]:
alidation_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)
validation_generator = validation_datagen.flow_from_directory(
        '/content/drive/MyDrive/EthioMedScan/dataset/LC_DataSet/valid',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=128,
        class_mode="sparse"
)

In [ ]:
est_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/EthioMedScan/dataset/LC_DataSet/test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=64,
        class_mode="sparse"
)

In [ ]:
for image_batch, label_batch in test_generator:
    print(image_batch[0])
    break

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Define constants
IMAGE_SIZE = 256
CHANNELS = 3
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 10

# Define the model
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

# Define the optimizer and learning rate
learning_rate = 0.001
model.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    batch_size=64,
    validation_data=validation_generator,
    verbose=1,
    epochs=50,
)




In [ ]:
# Evaluate the model on the validation set
validation_loss, validation_accuracy = model.evaluate(validation_generator, verbose=0)

# Predict the labels for the validation set
predictions = model.predict(validation_generator)
predicted_labels = tf.math.argmax(predictions, axis=1)

# Calculate metrics
accuracy = accuracy_score(validation_generator.labels, predicted_labels)
precision = precision_score(validation_generator.labels, predicted_labels, average='macro')
recall = recall_score(validation_generator.labels, predicted_labels, average='macro')
f1 = f1_score(validation_generator.labels, predicted_labels, average='macro')
roc_auc = roc_auc_score(validation_generator.labels, predictions, multi_class='ovr')

# Print the results
print('Validation Loss:', validation_loss)
print('Validation Accuracy:', validation_accuracy)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-Score:', f1)
print('ROC AUC:', roc_auc)

# Plot the training and validation losses
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracies
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Compute the confusion matrix
cm = confusion_matrix(validation_generator.labels, predicted_labels)

# Plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(validation_generator.class_indices.keys()))
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
#save Yuur model
model.save("/content/drive/MyDrive/EthioMedScan/SavedModels/Lungcancer1.h5")


In [ ]:
model.summary()